In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import cv2


def get_frames(video_path: Path, img_size=None, n_frames=None):
    cpr = cv2.VideoCapture(video_path.as_posix())
    has_frame = True
    frames = []

    while has_frame:
        has_frame, frame = cpr.read()
        if has_frame:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if img_size:
                frame = cv2.resize(frame, img_size)
            frames.append(frame)
    cpr.release()
    return np.array(frames)


In [ ]:
data_dir = Path("../data/sibur_data/")
videos_paths = list(data_dir.rglob("*.mp4"))
print(len(videos_paths))

clips = [
    get_frames(vp)
    for vp in tqdm(videos_paths[:50])
]

masks = [
    np.ones(len(c), dtype=bool)
    for c in clips
]

new_masks = []
for m in masks:
    new_masks.append(m)
    for _ in range(5):
        new_m = m.copy()
        new_m[np.random.choice(range(0, len(m)), int(len(m)*0.4), replace=False).astype(int)] = False
        new_masks.append(new_m)

masked_clips = []
for i, c in enumerate(clips):
    for m in new_masks[i*6:(i+1)*6]:
        masked_clips.append(c[m])

In [ ]:
import sys
sys.path.append("../")
from submit_videorec.predict import predict

In [ ]:
preds1 = [
    predict(clip)
    for clip in masked_clips
]